In [ ]:
form_one = r'\$\d+\.?\d*\s*[mb]illion'

In [ ]:
box_office.str.contains(form_one, flags=re.IGNORECASE, na=False).sum()

In [ ]:
form_two = r'\$\d{1,3}(?:,\d{3})+'

box_office.str.contains(form_two, flags=re.IGNORECASE, na=False).sum()

In [ ]:
matches_form_one = box_office.str.contains(form_one, flags=re.IGNORECASE, na=False)

matches_form_two = box_office.str.contains(form_two, flags=re.IGNORECASE, na=False)

In [ ]:
# this will throw an error!
box_office[(not matches_form_one) and (not matches_form_two)]

In [ ]:
box_office[~matches_form_one & ~matches_form_two]

In [ ]:
form_one = r'\$\s*\d+\.?\d*\s*[mb]illi?on'

form_two = r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)'

In [ ]:
box_office = box_office.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)

In [ ]:
box_office.str.extract(f'({form_one}|{form_two})')

def parse_dollars(s):
    
    # if s is not a string, return NaN
    
    if type(s) != str:
        
        return np.nan

    # if input is of the form $###.# million
    
    if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " million"
        
        s = re.sub('\$|\s|[a-zA-Z]','', s)

        # convert to float and multiply by a million
        
        value = float(s) * 10**6

        # return value
        
        return value

    # if input is of the form $###.# billion
    
    elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " billion"
        
        s = re.sub('\$|\s|[a-zA-Z]','', s)

        # convert to float and multiply by a billion
        
        value = float(s) * 10**9

        # return value
        
        return value

    # if input is of the form $###,###,###
    
    elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):

        # remove dollar sign and commas
        
        s = re.sub('\$|,','', s)

        # convert to float
        
        value = float(s)

        # return value
        
        return value

    # otherwise, return NaN
    
    else:
        
        return np.nan
        

In [ ]:
budget = wiki_movies_df['Budget'].dropna()

budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)

budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)

matches_form_one = budget.str.contains(form_one, flags=re.IGNORECASE, na=False)

matches_form_two = budget.str.contains(form_two, flags=re.IGNORECASE, na=False)

budget[~matches_form_one & ~matches_form_two]

In [ ]:
wiki_movies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)

wiki_movies_df.drop('Budget', axis=1, inplace=True)

release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)

In [ ]:
date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]?\d,\s\d{4}'

date_form_two = r'\d{4}.[01]\d.[0123]\d'

date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'

date_form_four = r'\d{4}'

In [ ]:
release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})', flags=re.IGNORECASE)

In [ ]:
wiki_movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)

running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)

In [ ]:
running_time.str.contains(r'^\d*\s*minutes$', flags=re.IGNORECASE, na=False).sum()

running_time[running_time.str.contains(r'^\d*\s*minutes$', flags=re.IGNORECASE, na=False) != True]

In [ ]:
running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)

In [ ]:
wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)

In [ ]:
wiki_movies_df.drop('Running time', axis=1, inplace=True)

In [ ]:
kaggle_metadata['adult'].value_counts()

In [ ]:
kaggle_metadata[~kaggle_metadata['adult'].isin(['True','False'])]

In [ ]:
kaggle_metadata = kaggle_metadata[kaggle_metadata['adult'] == 'False'].drop('adult',axis='columns')

In [ ]:
kaggle_metadata['video'].value_counts()

In [ ]:
kaggle_metadata['video'] = kaggle_metadata['video'] == 'True'

In [ ]:
kaggle_metadata['budget'] = kaggle_metadata['budget'].astype(int)

kaggle_metadata['id'] = pd.to_numeric(kaggle_metadata['id'], errors='raise')

kaggle_metadata['popularity'] = pd.to_numeric(kaggle_metadata['popularity'], errors='raise')

In [ ]:
ratings.info(null_counts=True)

In [ ]:
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')

In [ ]:
pd.options.display.float_format = '{:20,.2f}'.format

ratings['rating'].plot(kind='hist')

ratings['rating'].describe()